## Discover 802.15.4

#### Radio settings

If you are running this training at the same time as other people on the testbed, it is a good idea to change the default radio configuration to avoid too much collision with others.

Use the following cell to give you random values for channel and PAN ID.

In [1]:
import os,binascii,random
pan_id = binascii.b2a_hex(os.urandom(2)).decode()
channel = random.randint(11, 26)
print('Use CHANNEL={}, PAN_ID=0x{}'.format(channel, pan_id))

Use CHANNEL=26, PAN_ID=0xe86f


For these values to be taken into account you can set an environment variable for the notebook. Modify the values in the cell below with those obtained and execute it.

In [2]:
%env CHANNEL=26

env: CHANNEL=26


In [3]:
%env PAN_ID=0xE86F

env: PAN_ID=0xE86F


### Compile the RIOT firmware

Here you will use the `default` example from RIOT.

Let's define a environment variable to shorten the command lines.

In [1]:
%env APP_DIR = ../../RIOT/examples/default

env: APP_DIR=../../RIOT/examples/default


Compile the example:

In [2]:
!make -C $APP_DIR BOARD=iotlab-m3 DEFAULT_CHANNEL=$CHANNEL DEFAULT_PAN_ID=$PAN_ID

make: Entering directory '/home/jovyan/work/iot-lab-training/riot/RIOT/examples/default'
Building application "default" for "iotlab-m3" with MCU "stm32f1".

"make" -C /home/jovyan/work/iot-lab-training/riot/RIOT/boards/iotlab-m3
"make" -C /home/jovyan/work/iot-lab-training/riot/RIOT/boards/common/iotlab
"make" -C /home/jovyan/work/iot-lab-training/riot/RIOT/core
"make" -C /home/jovyan/work/iot-lab-training/riot/RIOT/cpu/stm32f1
"make" -C /home/jovyan/work/iot-lab-training/riot/RIOT/cpu/cortexm_common
"make" -C /home/jovyan/work/iot-lab-training/riot/RIOT/cpu/cortexm_common/periph
"make" -C /home/jovyan/work/iot-lab-training/riot/RIOT/cpu/stm32_common
"make" -C /home/jovyan/work/iot-lab-training/riot/RIOT/cpu/stm32_common/periph
"make" -C /home/jovyan/work/iot-lab-training/riot/RIOT/cpu/stm32f1/periph
"make" -C /home/jovyan/work/iot-lab-training/riot/RIOT/drivers
"make" -C /home/jovyan/work/iot-lab-training/riot/RIOT/drivers/at86rf2xx
"make" -C /home/jovyan/work/iot-lab-training/riot/RI

### Submit an experiment on IoT-LAB

1. Choose your site (grenoble|lille|strasbourg|...):

In [6]:
%env SITE = grenoble

env: SITE=grenoble


2. Submit an experiment with three nodes and the compiled firware:

In [7]:
!iotlab-experiment submit -d 60 -l 3,archi=m3:at86rf231+site=$SITE,$APP_DIR/bin/iotlab-m3/default.elf

{
    "id": 199005
}


3. Wait for the experiment to be in the Running state:

In [8]:
!iotlab-experiment wait --timeout 30 --cancel-on-timeout

Waiting that experiment 199005 gets in state Running
"Running"


**Note:** If the command above returns the message `Timeout reached, cancelling experiment <exp_id>`, try to re-submit your experiment later or try on another site.

4. Check the nodes allocated to the experiment

In [9]:
!iotlab-experiment get -ni

{
    "items": [
        {
            "grenoble": {
                "m3": "95-96+101"
            }
        }
    ]
}


From here you will interact with the nodes. To this end, you need to connect to node's serial link.

5. For each of the three nodes open a Jupyter terminal (use `File > New > Terminal`), use the following command replacing `<id>` and `<site>` with the right values:

### Send radio packets

1. Get the network config for each node (interface and address). 

```
ifconfig
Iface  4  HWaddr: 5C:22  Channel: 26  Page: 0  NID: 0x23
          Long HWaddr: 15:11:6B:10:65:FC:5C:22 
           TX-Power: 0dBm  State: IDLE  max. Retrans.: 3  CSMA Retries: 4 
          AUTOACK  ACK_REQ  CSMA  L2-PDU:102 Source address length: 2
```

  Here the node's radio interface (`Iface`) is _4_ and its long hardware address (`Long HWaddr`) is _15:11:6B:10:65:FC:5C:22_.

The `txtsnd` shell command of RIOT let's you send a radio packet 
  ```
  usage: txtsnd <if> [<L2 addr>|bcast] <data>
  ```
  - `<if>` is the network interface to use
  - `[<L2 addr>|bcast]` allows to specify a destination network address to send the packet in unicast or _bcast_ to send the packet in broadcast
  - `<data>` is the text message that will be contained in the payload

2. Send a packet from the first node in broadcast, specifying the radio interface of the sender and the broadcast mode:

On the two other nodes terminals you should see that the radio packet has been received:
```
PKTDUMP: data received:
~~ SNIP  0 - size:   5 byte, type: NETTYPE_UNDEF (0)
00000000  68  65  6C  6C  6F
~~ SNIP  1 - size:  12 byte, type: NETTYPE_NETIF (-1)
if_pid: 4  rssi: -55  lqi: 255
flags: BROADCAST 
src_l2addr: 5C:22
dst_l2addr: FF:FF
~~ PKT    -  2 snips, total size:  17 byte
```
What does this output tell you ?
- SNIP 0 = payload = 5 bytes `68  65  6C  6C  6F` for 5 characters ['h','e','l','l','o']
- SNIP 1 = header ?
  - `if_pid: 4  rssi: -55  lqi: 255` gives radio network interface informations: Received Signal Strength Indication (RSSI) and  the Link Quality Indicator (LQI)
  - `src_l2addr: 5C:22` gives the address of the sender
  - `dst12_addr: FF:FF` is the broadcast destination address, which corresponds well to the `flags: BROADCAST`


3. Send now a packet from the first node to the second. Specifying the radio interface of the sender and replace `<long_hwaddr>` with the address of the destination:

You should have a similar output in the destination node's terminal:
```
PKTDUMP: data received:
~~ SNIP  0 - size:   5 byte, type: NETTYPE_UNDEF (0)
00000000  68  65  6C  6C  6F
~~ SNIP  1 - size:  12 byte, type: NETTYPE_NETIF (-1)
if_pid: 4  rssi: -55  lqi: 255
flags: 0x0
src_l2addr: 5C:22
dst_l2addr: 22:4A
~~ PKT    -  2 snips, total size:  17 byte
```

- SNIP 0 remains the same since you send the same text message
- SNIP 1 contains the destination address, which correspond well to the node's address whom print this output

### Radio isolation

1. Replace `<channel> <id>`by the right values. For `<channel>` you have to choose a value between 11 and 26 different from the one obtained at the beginning of the notebook in `Radio settings` section. After run the following cell to recompile the example and flash the third node of your experiment:

In [12]:
!make -C $APP_DIR BOARD=iotlab-m3 DEFAULT_CHANNEL=16 DEFAULT_PAN_ID=$PAN_ID IOTLAB_NODE=m3-101.$SITE.iot-lab.info flash

make: Entering directory '/home/jovyan/work/iot-lab-training/riot/RIOT/examples/default'
Building application "default" for "iotlab-m3" with MCU "stm32f1".

"make" -C /home/jovyan/work/iot-lab-training/riot/RIOT/boards/iotlab-m3
"make" -C /home/jovyan/work/iot-lab-training/riot/RIOT/boards/common/iotlab
"make" -C /home/jovyan/work/iot-lab-training/riot/RIOT/core
"make" -C /home/jovyan/work/iot-lab-training/riot/RIOT/cpu/stm32f1
"make" -C /home/jovyan/work/iot-lab-training/riot/RIOT/cpu/cortexm_common
"make" -C /home/jovyan/work/iot-lab-training/riot/RIOT/cpu/cortexm_common/periph
"make" -C /home/jovyan/work/iot-lab-training/riot/RIOT/cpu/stm32_common
"make" -C /home/jovyan/work/iot-lab-training/riot/RIOT/cpu/stm32_common/periph
"make" -C /home/jovyan/work/iot-lab-training/riot/RIOT/cpu/stm32f1/periph
"make" -C /home/jovyan/work/iot-lab-training/riot/RIOT/drivers
"make" -C /home/jovyan/work/iot-lab-training/riot/RIOT/drivers/at86rf2xx
"make" -C /home/jovyan/work/iot-lab-training/riot/RI

2. Send a new broadcast message from the first node

This time, your third node has not received the packet, since its firmware is using another channel and is no more able to communicate with nodes using a different one.

3. Call the `ifconfig` command in each terminal to see channel used by each node.

### Free up the resources

Since you finished the training, stop your experiment to free up the experiment nodes:

In [13]:
!iotlab-experiment stop

{
    "id": 199005,
    "status": "Delete request registered"
}


The serial link connection through SSH will be closed automatically.